## 0. Introduction
Objectif: Fine-tuner le modèle XGBoost en explorant différents hyperparamètres.

Traçabilité et reproductibilité via MLflow.

## 1. Import & configuration

In [ ]:
import sys, os
import mlflow
import mlflow.xgboost
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (
    RocCurveDisplay,
    PrecisionRecallDisplay,
)
import seaborn as sns
from matplotlib import pyplot as plt
from xgboost import XGBClassifier
import pandas as pd
from rich import print

# Dataset + constantes
sys.path.insert(0, "..")
from config import PROCESSED_DATA_PATH, TARGET_COL, SEED, FIG_DIR, GLOBAL_SCORING, CV_STRATEGY
from utils import compute_train_test_metrics

df = pd.read_csv(PROCESSED_DATA_PATH)
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]

# --- Identifier les colonnes numériques et binaires
numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns.tolist()
binary_cols = [col for col in numeric_cols if X[col].nunique() == 2 or (X[col].dtype == 'int64' and set(X[col].unique()) == {0, 1})]
numeric_cols = [col for col in numeric_cols if col not in binary_cols]

NUMERIC_COLS = numeric_cols
BINARY_COLS = binary_cols

print(f"[bold cyan]📊 Colonnes détectées par feature engineering:[/bold cyan]")
print(f"   - Numériques: {NUMERIC_COLS}")
print(f"   - Binaires: {BINARY_COLS}")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=SEED
)

os.environ["MLFLOW_ENABLE_SYSTEM_METRICS_LOGGING"] = "false"
mlflow.autolog(disable=True)
mlflow.end_run()
mlflow.set_tracking_uri("file:../mlruns")
mlflow.set_experiment("Rugby Kicks Classification - XGBoost Finetuning")
mlflow.set_tag("author", "Xavier")
mlflow.set_tag("project", "OC P6")

X_trainfull = pd.concat([X_train, y_train], axis=1)
X_testfull = pd.concat([X_test, y_test], axis=1)

train_dataset = mlflow.data.from_pandas(X_trainfull, source=PROCESSED_DATA_PATH, name="Training_set")
test_dataset = mlflow.data.from_pandas(X_testfull, source=PROCESSED_DATA_PATH, name="Test_set")

if mlflow.active_run():
    print("⚠️ Run actif trouvé :", mlflow.active_run().info.run_id)
    mlflow.end_run()

print("[bold green]✅ Configuration MLflow prête[/bold green]")

✅ Config initialisée depuis : /Users/xaviercoulon/Documents/OC/OC_P6_Rugby_MLOps


📊 Colonnes détectées par feature engineering:

- Numériques: ['time_norm', 'distance', 'angle', 'wind_speed', 'precipitation_probability']

- Binaires: ['is_left_footed', 'game_away', 'is_endgame', 'is_start', 'is_left_side', 'has_previous_attempts']

2025/11/13 15:59:13 INFO mlflow.tracking.fluent: Experiment with name 'Rugby Kicks Classification - XGBoost Finetuning' does not exist. Creating a new experiment.
/Users/xaviercoulon/Documents/OC/OC_P6_Rugby_MLOps/.venv/lib/python3.12/site-packages/mlflow/data/dataset_source_registry.py:148: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for '/Users/xaviercoulon/Documents/OC/OC_P6_Rugby_MLOps/data/processed/kicks_ready_for_model.csv'. Exception: 
  return _dataset_source_registry.resolve(
/Users/xaviercoulon/Documents/OC/OC_P6_Rugby_MLOps/.venv/lib/python3.12/site-packages/mlflow/data/dataset_source_registry.py:148: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
/Users/xaviercoulon/Documents/OC/OC_P6_Rugby_MLOps/.venv/lib/python3.12/site-packa

⚠️ Run actif trouvé : 9ea75cda27cd4e3dacad1a266443efba

✅ Configuration MLflow prête

## 2. Preprocessing & Feature Scaling

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Créer la pipeline de preprocessing
preprocessing_pipeline = ColumnTransformer([
    ('scaler', StandardScaler(), NUMERIC_COLS),
    ('passthrough', 'passthrough', BINARY_COLS)
], remainder='drop')

# FIT sur X_train UNIQUEMENT (prevent data leakage!)
X_train_processed = preprocessing_pipeline.fit_transform(X_train)
X_test_processed = preprocessing_pipeline.transform(X_test)

# Convertir en DataFrame pour maintenabilité
feature_names = NUMERIC_COLS + BINARY_COLS
X_train_processed = pd.DataFrame(X_train_processed, columns=feature_names, index=X_train.index)
X_test_processed = pd.DataFrame(X_test_processed, columns=feature_names, index=X_test.index)

print("[bold green]✅ Pipeline de preprocessing créée et appliquée[/bold green]")
print(f"   - Colonnes numériques standardisées: {NUMERIC_COLS}")
print(f"   - Colonnes binaires conservées: {BINARY_COLS}")
print(f"   - X_train shape: {X_train_processed.shape}")
print(f"   - X_test shape: {X_test_processed.shape}")

✅ Pipeline de preprocessing créée et appliquée

- Colonnes numériques standardisées: ['time_norm', 'distance', 'angle', 'wind_speed', 
'precipitation_probability']

- Colonnes binaires conservées: ['is_left_footed', 'game_away', 'is_endgame', 'is_start', 'is_left_side', 
'has_previous_attempts']

- X_train shape: (13873, 11)

- X_test shape: (3469, 11)

## 3. XGBoost Fine-tuning with GridSearchCV

In [3]:
# Calcul du poids de la classe majoritaire / minoritaire
neg, pos = np.bincount(y_train)
scale_pos_weight = neg / pos

# --- Hyperparamètres de base (à partir du modèle benchmark)
base_params = {
    "scale_pos_weight": scale_pos_weight,
    "random_state": SEED,
    "eval_metric": "logloss",
    "n_jobs": -1,
}

# --- Grille de paramètres à explorer
param_grid = {
    "n_estimators": [200, 300, 400],
    "max_depth": [4, 5, 6],
    "learning_rate": [0.01, 0.05, 0.1],
    "subsample": [0.7, 0.8, 0.9],
    "colsample_bytree": [0.7, 0.8, 0.9],
    "min_child_weight": [2, 3, 5],
}

print("[bold cyan]🔍 Grille de paramètres pour GridSearchCV:[/bold cyan]")
for param, values in param_grid.items():
    print(f"   - {param}: {values}")
print(f"\n   Total combinations: {np.prod([len(v) for v in param_grid.values()])} (will use CV={CV_STRATEGY.get_n_splits()})") 

🔍 Grille de paramètres pour GridSearchCV:

- n_estimators: [200, 300, 400]

- max_depth: [4, 5, 6]

- learning_rate: [0.01, 0.05, 0.1]

- subsample: [0.7, 0.8, 0.9]

- colsample_bytree: [0.7, 0.8, 0.9]

- min_child_weight: [2, 3, 5]

Total combinations: 729 (will use CV=5)

In [4]:
with mlflow.start_run(
    run_name="XGBoost_GridSearchCV",
    description="Fine-tuning XGBoost avec GridSearchCV - exhaustive search",
):

    mlflow.log_input(train_dataset, context="training")
    mlflow.log_input(test_dataset, context="testing")
    mlflow.set_tag("model_type", "xgboost_finetuned")
    mlflow.set_tag("optimization_method", "gridsearchcv")

    # --- Logger les paramètres de base
    base_params_log = base_params.copy()
    base_params_log["scale_pos_weight"] = round(scale_pos_weight, 2)
    mlflow.log_params({f"base_{k}": str(v) for k, v in base_params_log.items()})
    
    # --- Logger la grille de paramètres
    for param, values in param_grid.items():
        mlflow.log_param(f"grid_{param}", str(values))

    # --- GridSearchCV
    xgb_base = XGBClassifier(**base_params)
    grid_search = GridSearchCV(
        estimator=xgb_base,
        param_grid=param_grid,
        cv=CV_STRATEGY,
        scoring="f1_weighted",  # Utiliser F1 pondéré pour imbalanced data
        n_jobs=-1,
        verbose=1
    )

    print("[bold cyan]🔄 Lancement de GridSearchCV... (cela peut prendre du temps)[/bold cyan]")
    grid_search.fit(X_train_processed, y_train)

    # --- Récupérer le meilleur modèle
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_
    best_cv_score = grid_search.best_score_

    print(f"[bold green]✅ GridSearchCV terminé[/bold green]")
    print(f"   - Best CV Score (F1): {best_cv_score:.4f}")
    print(f"[bold yellow]🏆 Best Parameters found:[/bold yellow]")
    for param, value in best_params.items():
        print(f"   - {param}: {value}")
    
    # --- Logger les meilleurs paramètres
    mlflow.log_params({f"best_{k}": str(v) for k, v in best_params.items()})
    mlflow.log_metric("best_cv_score_f1_weighted", best_cv_score)

    # --- Prédictions finales
    y_pred_train = best_model.predict(X_train_processed)
    y_proba_train = best_model.predict_proba(X_train_processed)[:, 1]
    
    y_pred = best_model.predict(X_test_processed)
    y_proba = best_model.predict_proba(X_test_processed)[:, 1]

    # --- Métriques train/test + overfitting gaps
    final_metrics_test, cm, (fp_rate, fn_rate) = compute_train_test_metrics(
        y_train, y_pred_train, y_proba_train,
        y_test, y_pred, y_proba
    )
    
    # --- Log metrics
    mlflow.log_metrics(final_metrics_test)

    # --- Log du modèle
    mlflow.xgboost.log_model(
        xgb_model=best_model,
        name="xgboost_finetuned_model",
        input_example=X_test_processed.iloc[:5],
    )

    # --- Matrice de confusion
    plt.figure(figsize=(4, 4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="mako", cbar=False)
    plt.title("Confusion Matrix - XGBoost Finetuned")
    plt.xlabel("Prédictions")
    plt.ylabel("Vérités terrain")
    plt.tight_layout()
    cm_path = f"{FIG_DIR}/xgb_finetuned_confusion_matrix.png"
    plt.savefig(cm_path)
    mlflow.log_artifact(cm_path)
    plt.close()

    # --- Courbe ROC
    RocCurveDisplay.from_estimator(best_model, X_test_processed, y_test)
    plt.title("ROC Curve - XGBoost Finetuned")
    plt.tight_layout()
    roc_path = f"{FIG_DIR}/xgb_finetuned_roc_curve.png"
    plt.savefig(roc_path)
    mlflow.log_artifact(roc_path)
    plt.close()

    # --- Courbe Precision-Recall
    PrecisionRecallDisplay.from_estimator(best_model, X_test_processed, y_test)
    plt.title("Precision-Recall Curve - XGBoost Finetuned")
    plt.tight_layout()
    pr_path = f"{FIG_DIR}/xgb_finetuned_pr_curve.png"
    plt.savefig(pr_path)
    mlflow.log_artifact(pr_path)
    plt.close()

    # --- Importance des features
    importances = pd.Series(best_model.feature_importances_, index=X.columns).sort_values(
        ascending=False
    )
    plt.figure(figsize=(8, 5))
    imp_df = pd.DataFrame({'importance': importances[:10].values, 'feature': importances[:10].index})
    sns.barplot(data=imp_df, x='importance', y='feature', palette="rocket", hue='feature', legend=False)
    plt.title("Top 10 Feature Importances - XGBoost Finetuned")
    plt.tight_layout()
    feat_imp_path = f"{FIG_DIR}/xgb_finetuned_feature_importances.png"
    plt.savefig(feat_imp_path)
    mlflow.log_artifact(feat_imp_path)
    plt.close()

    print("[bold cyan]✅ XGBoost Finetuned logué dans MLflow avec succès[/bold cyan]")

# 🔎 Résumé local
print(
    f"""
📊 Résumé des métriques - XGBoost Finetuned :
  Train Accuracy: {final_metrics_test['train_accuracy']:.3f}
  Test Accuracy:  {final_metrics_test['final_test_accuracy']:.3f}
  Overfitting Gap (Accuracy): {final_metrics_test['overfit_gap_accuracy']:.3f}
  ---
  Train F1 classe 0: {final_metrics_test['train_f1_0']:.3f}
  Test F1 classe 0:  {final_metrics_test['final_test_f1_0']:.3f}
  Overfitting Gap (F1_0): {final_metrics_test['overfit_gap_f1_0']:.3f}
  ---
  Accuracy:  {final_metrics_test['final_test_accuracy']:.3f}
  ROC-AUC:   {final_metrics_test['final_test_auc']:.3f}
  Precision classe 0: {final_metrics_test['final_test_precision_0']:.3f}
  Recall classe 0:    {final_metrics_test['final_test_recall_0']:.3f}
  F1-score classe 0:  {final_metrics_test['final_test_f1_0']:.3f}
  Precision classe 1: {final_metrics_test['final_test_precision_1']:.3f}
  Recall classe 1:    {final_metrics_test['final_test_recall_1']:.3f}
  F1-score classe 1:  {final_metrics_test['final_test_f1_1']:.3f}
  FPR:       {fp_rate:.3f}
  FNR:       {fn_rate:.3f}
"""
)

/Users/xaviercoulon/Documents/OC/OC_P6_Rugby_MLOps/.venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


🔄 Lancement de GridSearchCV... (cela peut prendre du temps)

Fitting 5 folds for each of 729 candidates, totalling 3645 fits


✅ GridSearchCV terminé

- Best CV Score (F1): 0.7209

...

- n_estimators: 400

- subsample: 0.8

/Users/xaviercoulon/Documents/OC/OC_P6_Rugby_MLOps/.venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
/Users/xaviercoulon/Documents/OC/OC_P6_Rugby_MLOps/.venv/lib/python3.12/site-packages/xgboost/sklearn.py:1115: UserWarning: [16:00:25] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1575

✅ XGBoost Finetuned logué dans MLflow avec succès

📊 Résumé des métriques - XGBoost Finetuned :
  Train Accuracy: 0.831
  Test Accuracy:  0.709
  Overfitting Gap (Accuracy): 0.123
  ---
  Train F1 classe 0: 0.727
  Test F1 classe 0:  0.509
  Overfitting Gap (F1_0): 0.217
  ---
  Accuracy:  0.709
  ROC-AUC:   0.769
  Precision classe 0: 0.423
  Recall classe 0:    0.641
  F1-score classe 0:  0.509
  Precision classe 1: 0.868
  Recall classe 1:    0.730
  F1-score classe 1:  0.793
  FPR:       0.359
  FNR:       0.270